# MLlib 실제로 사용하기
- 이진 분류 모델 만들어보기

## 데이터 구성
- color: 범주형 변수
- lab: good or bad로 구성된 범주형 레이블(y)
- value1,value2: 수치형 변수

In [0]:
path= '/FileStore/tables/bin/simple-ml'
df = spark.read.json(path)

In [0]:
display(df.orderBy('value2'))

color,lab,value1,value2
green,good,12,14.386294994851129
red,bad,2,14.386294994851129
red,bad,16,14.386294994851129
green,good,1,14.386294994851129
red,good,35,14.386294994851129
red,bad,2,14.386294994851129
red,bad,16,14.386294994851129
green,good,1,14.386294994851129
blue,bad,8,14.386294994851129
blue,bad,12,14.386294994851129


## 변환자를 사용해서 피처 엔지니어링 수행하기 
- 변환자는 여러 방식으로 현재 컬럼을 조작하는 데 사용됨
  - 특징 수를 줄이거나
  - 더 많은 특징을 추가하거나
  - 현재 특징을 변형하는 등
- MLlib에서 제공하는 대부분의 알고리즘은 입력변수가 <strong>Double타입(레이블용)이거나 Vector[Double]타입(특징용)</strong>으로 구성되어야함

### RFormula

In [0]:
from pyspark.ml.feature import RFormula
supervised = RFormula(formula = "lab~.+color:value1 + color:value2")

------
- ~ -> 함수에서 타깃과 항 분리
- \+ -> 연결 기호로, '+0'은 절편 제거
- \- -> 삭제 기호로, '-1'은 절편 제거('+0'과 결과 동일)
- : -> 상호작용(수치형 값이나 이진화된 범주 값에 대한 곱셈)
- . -> 종속변수를 제외한 모든 컬럼


- 즉, 위의 수식의 의미는
  - ~ 으로 lab(종속변수)과 독립변수를 구분했고
  - . 으로 모든 독립변수를 사용할 것이라 선언했고
  - : 로 두 변수의 상호작용을 고려한 변수를 정의하고
  - \+ 로 새로운 변수를 추가했다


[참고](https://lovetoken.github.io/r/2016/12/06/formula_usage.html)

#### fit
- fit 메서드를 호출하면 실제로 데이터를 변형시키는 데 사용할 수 있는 <strong>'학습된' 버전의 변환자</strong>를 리턴함
  - 이 학습된 변환자는 항상 Model이라는 단어와 함께 쓰임
- RFormula는 fit 함수가 호출되는 과정에서
  - 데이터를 검사하고
  - 지정된 수식에 따라 데이터를 변환할 객체인 RformulaModel을 리턴함 
- 학습된 변환자를 사용하면 스파크는 <strong>자동으로 범주형 변수를 Double 타입으로 변환함</strong>
  - 예제에서 color 컬럼의 각 범주에 숫자값을 할당
  - color:value1, color:value2 상호작용항을 만들고 모두 단일 벡터에 넣음

In [0]:
fittedRF = supervised.fit(df)
preparedRF = fittedRF.transform(df)
preparedRF.show()

+-----+----+------+------------------+--------------------+-----+
color| lab|value1| value2| features|label|
+-----+----+------+------------------+--------------------+-----+
green|good| 1|14.386294994851129|(10,[1,2,3,5,8],[...| 1.0|
 blue| bad| 8|14.386294994851129|(10,[2,3,6,9],[8....| 0.0|
 blue| bad| 12|14.386294994851129|(10,[2,3,6,9],[12...| 0.0|
green|good| 15| 38.97187133755819|(10,[1,2,3,5,8],[...| 1.0|
green|good| 12|14.386294994851129|(10,[1,2,3,5,8],[...| 1.0|
green| bad| 16|14.386294994851129|(10,[1,2,3,5,8],[...| 0.0|
 red|good| 35|14.386294994851129|(10,[0,2,3,4,7],[...| 1.0|
 red| bad| 1| 38.97187133755819|(10,[0,2,3,4,7],[...| 0.0|
 red| bad| 2|14.386294994851129|(10,[0,2,3,4,7],[...| 0.0|
 red| bad| 16|14.386294994851129|(10,[0,2,3,4,7],[...| 0.0|
 red|good| 45| 38.97187133755819|(10,[0,2,3,4,7],[...| 1.0|
green|good| 1|14.386294994851129|(10,[1,2,3,5,8],[...| 1.0|
 blue| bad| 8|14.386294994851129|(10,[2,3,6,9],[8....| 0.0|
 blue| bad| 12|14.386294994851129|(10,[2,3,6,9],[12...| 0.0|
green|good| 15| 38.97187133755819|(10,[1,2,3,5,8],[...| 1.0|
green|good| 12|14.386294994851129|(10,[1,2,3,5,8],[...| 1.0|
green| bad| 16|14.386294994851129|(10,[1,2,3,5,8],[...| 0.0|
 red|good| 35|14.386294994851129|(10,[0,2,3,4,7],[...| 1.0|
 red| bad| 1| 38.97187133755819|(10,[0,2,3,4,7],[...| 0.0|
 red| bad| 2|14.386294994851129|(10,[0,2,3,4,7],[...| 0.0|
+-----+----+------+------------------+--------------------+-----+
only showing top 20 rows

### 데이터 분할
- 좋은 테스트셋을 확보하는 것은 실무에서 활용할 수 있는 좋은 모델을 학습시키기 위한 가장 중요한 절차
- 만약 하이퍼파라미터 튜닝을 위해 테스트셋을 사용하지 않거나 대표성 있는 테스트셋을 생성하지 않는다면 좋은 모델이 나올 수 없음

In [0]:
train ,test = preparedRF.randomSplit([0.7,0.3])

## 추정자 
- 이제는 모델을 적합시킬 차례
- 분류기를 생성하기위해 알고리즘을 객체화하고, 레이블 컬럼과 특징 컬럼을 설정함
- 스파크 MLlib의 모든 추정자에 대한 기본 명칭
  - 레이블 컬럼의 이름은 'label'
  - 특징 컬럼의 이름은 'features'
- explainParams 메서드로 설정된 모든 파라미터의 설명이 출력
  - MLlib에서 제공하는 모든 알고리즘에서 사용 가능

In [0]:
from pyspark.ml.classification import LogisticRegression
lr= LogisticRegression(labelCol='label', featuresCol= 'features')

In [0]:
lr.explainParams()

Out[ 12 ]: "aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)\nelasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)\nfamily: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)\nfeaturesCol: features column name. (default: features, current: features)\nfitIntercept: whether to fit an intercept term. (default: True)\nlabelCol: label column name. (default: label, current: label)\nlowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)\nlowerBoundsOnIntercepts: The lower bounds on intercepts if fitting under bound constrained optimization. The bounds vector size must beequal with 1 for binomial regression, or the number oflasses for multinomial regression. (undefined)\nmaxIter: max number of iterations (>= 0). (default: 100)\npredictionCol: prediction column name. (default: prediction)\nprobabilityCol: Column name for predicted class conditional probabilities. Note: Not all models output well-calibrated probability estimates! These probabilities should be treated as confidences, not precise probabilities. (default: probability)\nrawPredictionCol: raw prediction (a.k.a. confidence) column name. (default: rawPrediction)\nregParam: regularization parameter (>= 0). (default: 0.0)\nstandardization: whether to standardize the training features before fitting the model. (default: True)\nthreshold: Threshold in binary classification prediction, in range [0, 1]. If threshold and thresholds are both set, they must match.e.g. if threshold is p, then thresholds must be equal to [1-p, p]. (default: 0.5)\nthresholds: Thresholds in multi-class classification to adjust the probability of predicting each class. Array must have length equal to the number of classes, with values > 0, excepting that at most one value may be 0. The class with largest value p/t is predicted, where p is the original probability of that class and t is the class's threshold. (undefined)\ntol: the convergence tolerance for iterative algorithms (>= 0). (default: 1e-06)\nupperBoundsOnCoefficients: The upper bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)\nupperBoundsOnIntercepts: The upper bounds on intercepts if fitting under bound constrained optimization. The bound vector size must be equal with 1 for binomial regression, or the number of classes for multinomial regression. (undefined)\nweightCol: weight column name. If this is not set or empty, we treat all instance weights as 1.0. (undefined)"

### fit
- 변환자와 달리 머신러닝 모델을 적합시키는 것은 '사전학습' 개념임
- 이 단계까지 완료되면 모델을 사용할 수 있음
- 예측은 transform메서드로 적용

In [0]:
fittedLR = lr.fit(train)

In [0]:
fittedLR.transform(train).select('label', 'prediction').show()

+-----+----------+
label|prediction|
+-----+----------+
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 1.0| 1.0|
 1.0| 1.0|
 1.0| 1.0|
+-----+----------+
only showing top 20 rows

## 워크플로를 파이프라인으로 만들기 
- 분석과정에서 많은 변환을 실행하면 모든 단계를 개발하고 DataFrame을 추적하는 일이 매우 지루해짐
  - 이것이 스파크가 Pipeline 개념을 도입한 이유
- 파이프라인을 사용하면 마지막 단계인 추정자는 조건에 따라 자동으로 조정되며, 그에 따른 데이터 변환 과정을 설정할 수 있음
- 주의
  - <strong>변환자 객체나 모델 객체가 다른 파이프라인에서 재사용되지 않게 해야함</strong>
    - 다른 파이프라인을 생성하기 전에 항상 새로운 모델 객체를 만들어야함
  - 모델의 과적합을 방지하기 위해 홀드아웃 테스트셋을 생성하고 <strong>검증셋을 기반으로 하이퍼파라미터를 조정</strong>해야함
    - 검증셋을 생성할 땐 이전 단계에서 사용된 preparedDF가 아닌 원시 데이터를 기반으로 해야함

In [0]:
train,test = df.randomSplit([0.7,0.3])

In [0]:
#두 개의 추정자 생성
rForm = RFormula() #입력 특징의 타입 이해 + 새로운 특징 생성을 위해 형상 변형
lr = LogisticRegression().setLabelCol('label').setFeaturesCol('features') # 모델 생성을 위해 학습하는 알고리즘

### 파이프라인 적용

In [0]:
from pyspark.ml import Pipeline
stages = [rForm, lr]
pipeline = Pipeline().setStages(stages)

## 모델 학습 및 평가 
- 스파크에서 테스트할 다양한 하이퍼파라미터의 조합을 지정해 다양한 모델을 학습시킬 것임
- 이후 평가기를 사용하여 검증셋으로 각 모델의 예측 결과를 비교하여 최적의 모델을 선택할 것임

In [0]:
from pyspark.ml.tuning import ParamGridBuilder

In [0]:
params = ParamGridBuilder()\
.addGrid(rForm.formula, ["lab ~ . + color:value1", "lab ~ . + color:value1+color:value2"])\
.addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])\
.addGrid(lr.regParam, [0.1,2.0]).build()

----
- 기본값을 변형한 세 가지 하이퍼파라미터 적용
  - 두 개의 서로 다른 버전의 RFormula
  - 세 개의 서로 다른 옵션의 ElasticNet 파라미터
  - 두 개의 서로 다른 옵션의 일반화 파라미터
- 따라서 총 12개의 서로 다른 파라미터 조합을 학습함(12가지 버전의 로지스틱 회귀 학습)

### 평가기
- 자동으로 여러 모델을 동일한 평가지표를 적용하여 객관적으로 비교할 수 있게함

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator()\
.setMetricName('areaUnderROC')\
.setRawPredictionCol('prediction')\
.setLabelCol('label')

----
- 다양한 하이퍼파라미터를 시험하고
- areaUnderROC 지표로 성능을 비교하여 모델 적합도를 측정하고 모델을 선택함

In [0]:
#데이터 분할도 자동으로해서 하이퍼파라미터 튜닝을 자동으로 수행하도록
from pyspark.ml.tuning import TrainValidationSplit
tvs = TrainValidationSplit()\
.setTrainRatio(0.75)\
.setEstimatorParamMaps(params)\
.setEstimator(pipeline)\
.setEvaluator(evaluator)

In [0]:
tvsFitted = tvs.fit(train)

/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

In [0]:
evaluator.evaluate(tvsFitted.transform(test))

Out[ 29 ]: 0.9230769230769231